<h1><span style="color:red">Generate Concepts from Images in Image Collection</span></h1>

### This sample notebook will read survey images and add concepts found in the images to a new version of  SuAVE survey

This notebook uses Clarifai API (clarifai.com). To process your images, please generate your own API key at the web site.

## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

## 2. Import libraries

In [ ]:
# common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import pandas as pd
pd.set_option('display.max_colwidth', 0)
    
import numpy as np
import panel as pn

pn.extension()
def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

# specific imports
import requests
import re
import glob, os
import csv

from clarifai_grpc.channel.clarifai_channel import ClarifaiChannel
from clarifai_grpc.grpc.api import service_pb2_grpc

stub = service_pb2_grpc.V2Stub(ClarifaiChannel.get_grpc_channel())

## 3. Read the survey file and navigate to full-size images

In [ ]:
df = panellibs.extract_data(absolutePath + csv_file)
dflen = len(df.columns)

localdzc = dzc_file.replace("https://maxim.ucsd.edu/dzgen/lib-staging-uploads","/lib-nfs/dzgen")
full_images_location = localdzc.replace("/content.dzc","/full_images/")


## 4. Specify Clarifai API key
<b><h3><span style="color:red">Important: </span>Specify your Clarifai API Key, App ID, Personal Access Token, and User ID, below.<br>You can get them at clarifai.com</h3></b>


In [ ]:
# Retrieve the API key

# import credentials file
import yaml
from pathlib import Path

try:
    with open(Path.home() / 'creds.yml', 'r') as ymlfile:
        cfg = yaml.safe_load(ymlfile)
        api_key=cfg['api_creds']['cfy1']
        app_ID=cfg['api_creds']['cfy_appid']
        pat_ID=cfg['api_creds']['cfy_pat']
        user_ID=cfg['api_creds']['cfy_uid']


except IOError:
    printmd("<b><span style='color:red;font-size:150%'>Credentials are unavailable or incorrect. CANNOT CONTINUE.</span></b>")
    printmd("<b><span style='color:red;font-size:150%'>Please register and obtain the credentials at clarifai.com.</span></b>")

if api_key == '' :
    print('Cannot proceed without an API Key')
else:
    os.environ["CLARIFAI_API_KEY"] = api_key
    os.environ["CLARIFAI_APP_ID"] = app_ID
#     printmd("<b><span style='color:red'>Your CLARIFAI API Key is: </span></b>" +os.environ["CLARIFAI_API_KEY"])
#     printmd("<b><span style='color:red'>Your CLARIFAI APP ID is: </span></b>" +os.environ["CLARIFAI_APP_ID"])



In [ ]:
from clarifai_grpc.grpc.api import service_pb2, resources_pb2
from clarifai_grpc.grpc.api.status import status_code_pb2

metadata = (("authorization", f"Key {pat_ID}"),)


## 5. Extract the images, from SuAVE (local to the system) or at URL

In [ ]:
# One can point to a local directory with images or to a list of images at a URL

URL_or_local = 'local'  # the case of SuAVE. This only affects the format of a CLARIFAI request 

set_of_files = glob.glob(full_images_location+"*.png")

printmd("<b><span style='color:red'>Count of items to process: </span></b>" + str(len(set_of_files)))


## 6. Run the classifier

In [ ]:
all_data = []   # here, we accummulate the generated concepts for each image
counter = 0

for im in set_of_files:
    counter += 1
    if URL_or_local == 'url':
        
        request = service_pb2.PostModelOutputsRequest(
        # This is the model ID of a publicly available General model. You may use any other public or custom model ID.
            model_id="general-image-recognition",
            user_app_id=resources_pb2.UserAppIDSet(user_id=user_ID, app_id=app_ID),
            inputs=[
                resources_pb2.Input(
                    data=resources_pb2.Data(image=resources_pb2.Image(url=im))
                    )
            ],
        )
        response = stub.PostModelOutputs(request, metadata=metadata)
        
        
    else:
        with open(im, "rb") as f:
            file_bytes = f.read()
        request = service_pb2.PostModelOutputsRequest(
        # This is the model ID of a publicly available General model. You may use any other public or custom model ID.
            model_id="general-image-recognition",
            user_app_id=resources_pb2.UserAppIDSet(user_id=user_ID,app_id=app_ID),
            inputs=[
                resources_pb2.Input(
                    data=resources_pb2.Data(image=resources_pb2.Image(base64=file_bytes))
                    )
            ],
        )
        response = stub.PostModelOutputs(request, metadata=metadata)
        
    if response.status.code != status_code_pb2.SUCCESS:
        print(response)
        raise Exception(f"Request failed, status code: {response.status}")


    file_data = {}
    file_data['#img'] = os.path.basename(im)[:-4]
    
    cons = response.outputs[0].data.concepts
    
    for i in range(len(cons)):  
        file_data['concept_'+str(i+1)] = cons[i].name
        file_data['value_'+str(i+1)] = cons[i].value
    all_data.append(file_data)
    print(str(counter)," ::  Processed file: ", os.path.basename(im))
# debugging:
    if counter == 3:
        break
printmd("<b><span style='color:red'>" + str(counter) + " images processed</span></b>")


## 7. Add concepts to dataframe

In [ ]:
# adding individual concept fields, as well as a single multiple-response column with all concepts, to a dataframe
newdf = pd.DataFrame(all_data).fillna('')
newdf = newdf[['#img', 
         'concept_1', 'value_1',
         'concept_2', 'value_2',
         'concept_3', 'value_3',
         'concept_4', 'value_4',
         'concept_5', 'value_5',
         'concept_6', 'value_6',
         'concept_7', 'value_7',
         'concept_8', 'value_8',
         'concept_9', 'value_9',
         'concept_10', 'value_10',
         'concept_11', 'value_11',
         'concept_12', 'value_12',
         'concept_13', 'value_13',
         'concept_14', 'value_14',
         'concept_15', 'value_15',
         'concept_16', 'value_16',
         'concept_17', 'value_17',
         'concept_18', 'value_18',
         'concept_19', 'value_19',
         'concept_20', 'value_20']]
multi =''
for i in range(20):
    if i == 19:
        multi += newdf['concept_'+str(i+1)]
    else:
        multi += newdf['concept_'+str(i+1)] +'|'

newdf['tags#multi'] = multi
df_merged = pd.merge(df, newdf, on='#img', how='outer')
printmd("<b><span style='color:red'>Created new dataframe </span></b>")


In [ ]:
# view the dataframe
with pd.option_context("display.max_columns", None):
    if any("geometry" in col for col in df_merged.columns):
        display(df_merged.drop(['geometry'],axis=1))
    else:
        display(df_merged)
    
 

## 8. Save the new version of CSV file, and give a name to new survey

In [ ]:
new_file = suaveint.save_csv_file(df_merged, absolutePath, csv_file)

In [ ]:
#Input survey name

from IPython.display import display
input_text = widgets.Text(placeholder='Enter Survey Name...')
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

## 9. Generate the survey and create survey URL

In [ ]:
suaveint.create_survey(survey_url,new_file, survey_name, dzc_file, user, csv_file, view, views)